In [1]:
import numpy as np

%matplotlib ipympl
%matplotlib inline
import matplotlib.pyplot as plt
from astropy.table import Table

I don't understand why you need to run this command twice to get an interactive matplotlib display

In [2]:
%matplotlib ipympl

In [3]:
import lsst.daf.persistence        as dafPersist
import lsst.afw.geom               as afwGeom
from matplotlib.colors import LogNorm
import lsst.afw.table              as afwTable
import lsst.meas.extensions.psfex.psfexPsfDeterminer

import lsst.afw.display            as afwDisplay
import lsst.afw.display.utils as afwDisplayUtils

/Users/naka.keigo/lsst_stack/python/miniconda3-4.3.21/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


#### Load the high-level "tasks" that process the pixels

In [4]:
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask

#### Setup the displays (by default an interface to ds9, which won't work in the LSP)

In [5]:
afwDisplay.setDefaultBackend("matplotlib")

## Create the task

In [6]:
config = CharacterizeImageTask.ConfigClass()
config.psfIterations = 1
config.measurePsf.psfDeterminer.name = "psfex"
config.measurePsf.psfDeterminer["psfex"].samplingSize = 1.0
config.measurePsf.psfDeterminer["psfex"].spatialOrder = 3
#print(config.measurePsf.psfDeterminer["psfex"])
charImageTask = CharacterizeImageTask(None, config=config)

## Time to process some data

#### Read the input data

In [7]:
butler = dafPersist.Butler("/Users/naka.keigo/Desktop/integ/INTEGRATION/rerun/integration/arc/")

In [8]:
dataId = dict(visit=31)
exposure = butler.get("calexp", visit=31, arm="r",spectrograph=1)
dataRef=butler.dataRef("calexp", visit=31, arm="r", spectrograph=1)

In [9]:
degrees = lsst.geom.degrees
cdMatrix = np.array([[1.0e-4, 0.0], [0.0, 1.0e-4]], dtype=float)
exposure.setWcs(lsst.afw.geom.makeSkyWcs(crval=lsst.geom.SpherePoint(0*degrees, 0*degrees),
                                            crpix=lsst.geom.Point2D(0.0, 0.0),
                                            cdMatrix=cdMatrix))

In [10]:
%matplotlib ipympl

#disp = afwDisplay.Display(reopenPlot=True)
disp = afwDisplay.Display(reopenPlot=False, fastMaskDisplay=True)
disp.scale('asinh', -20, 30, Q=8)
disp.mtv(exposure, title=dataId)

disp.zoom(8, 500, 1500);

FigureCanvasNbAgg()

### Process the pixels

##### Characterise the exposure (e.g. estimate the PSF)

In [11]:
from lsst.obs.base import ExposureIdInfo
result = charImageTask.detectMeasureAndEstimatePsf(exposure,ExposureIdInfo(), None)

In [12]:
np.median(result.background.getImage().array)

-0.011891423

In [13]:
#result = charImageTask.run(dataRef, exposure, doUnpersist=False)
#result = charImageTask.run(exposure)

In [14]:
for s in result.sourceCat:
    disp.dot('+', *s.getCentroid(), ctype='RED')

In [15]:
maskedImage=exposure.getMaskedImage()
psf = exposure.getPsf()
w, h = exposure.getDimensions()

In [16]:
%matplotlib ipympl

mos = afwDisplayUtils.Mosaic()

for y in np.linspace(0, h, 10):
    for x in np.linspace(0, w, 10):
        im = psf.computeImage(afwGeom.PointD(x, y)).convertF()
        mos.append(im)

disp2 = afwDisplay.Display(2, reopenPlot=True)
disp2.scale('asinh', 'zscale')
mos.makeMosaic(display=disp2);

FigureCanvasNbAgg()

In [36]:
###Analysis
s = result.sourceCat[50]
    
im = psf.computeImage(s.getCentroid())
box=exposure.getImage()[im.getBBox()] # .convertF()
psfplt= psf.computeImage(afwGeom.PointD(s.getCentroid())).convertF()

print("RHL", np.median(exposure.variance[im.getBBox()].array))

#Normalized by Area

normbox = (box.array)
normpsf = psfplt.array*s.getApFlux()

#box20=normbox[10:30,10:30]
#psf20=normpsf[10:30,10:30]

#var_20 = np.abs(normbox)+20
var_20 = exposure.variance[im.getBBox()].array

plt.figure(figsize=(6,5))
plt.imshow(normbox,origin='lower')
plt.colorbar()

plt.figure(figsize=(6,5))
plt.imshow(exposure.variance[im.getBBox()].array ,origin='lower')
plt.colorbar()

plt.figure(figsize=(6,5))
plt.imshow(normbox,origin='lower')
plt.colorbar()

plt.figure(figsize=(6,5))
plt.imshow((normbox-normpsf)/np.sqrt(var_20),origin='lower',cmap='bwr')
plt.colorbar()
plt.title(r"$\chi$")

plt.figure(figsize=(6,5))
plt.imshow((normbox-normpsf) - np.median((normbox-normpsf)),origin='lower',cmap='bwr')
plt.colorbar()
plt.title(r"residual data - model - background")


#init_lamda,std_init_lamda,init_removal_lamda,std_init_removal_lamda = residual_1D(box20, psf20, var_20)

RHL 4.2754


FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [19]:
shape = psf.computeShape()
shape.getIxx()

1.5872134105924476

In [20]:
[a.getGain() for a in exposure.getDetector()]

[1.24, 1.24, 1.27, 1.18, 1.26, 1.2, 1.24, 1.26]

In [21]:
sci_image_smaller=box20[:,6:13]
var_image_smaller=psf20[:,6:13]
residual_initial_smaller=sci_image_smaller-var_image_smaller

sci_inputimage_smaller=sci_image_smaller
Px=np.sum(sci_inputimage_smaller,axis=0)/np.sum(sci_inputimage_smaller)
var_inputimage_smaller=var_image_smaller
Pv=np.sum(var_inputimage_smaller,axis=0)/np.sum(var_inputimage_smaller)
   
# nominator
weighted_inputimage_smaller=sci_inputimage_smaller*Px/(1)
weighted_varimage_smaller=var_inputimage_smaller*Pv/(1)
# denominator
weights_array=np.ones((sci_inputimage_smaller.shape[0],sci_inputimage_smaller.shape[1]))*Px**2
weights_vararray=np.ones((var_inputimage_smaller.shape[0],var_inputimage_smaller.shape[1]))*Pv**2

init_lamda=np.array(list(map(np.sum, weighted_inputimage_smaller)))/(np.array(list(map(np.sum,weights_array))))
init_lamda_boxcar=np.array(list(map(np.sum, sci_inputimage_smaller)))

init_varlamda=np.array(list(map(np.sum, weighted_varimage_smaller)))/(np.array(list(map(np.sum,weights_vararray))))
init_varlamda_boxcar=np.array(list(map(np.sum, var_inputimage_smaller)))

# Equation 8.5 from Horne
var_f_std_lamda=1/np.abs(np.sum(np.array(Px**2/(var_inputimage_smaller)),axis=1))
std_init_lamda=np.sqrt(var_f_std_lamda)
std_init_lamda_boxcar=np.sqrt(np.abs(np.array(list(map(np.sum, var_inputimage_smaller)))))

#################################
# Equation 8 from Horne with modification from Robert abut variance for initial removal
# note that this uses profile from full thing, and not "residual profile"

# nominator
weighted_inputimage_smaller=residual_initial_smaller*Px/(1)
# denominator
weights_array=np.ones((residual_initial_smaller.shape[0],residual_initial_smaller.shape[1]))*Px**2

init_removal_lamda=np.array(list(map(np.sum, weighted_inputimage_smaller)))/(np.array(list(map(np.sum,weights_array))))
init_removal_lamda_boxcar=np.array(list(map(np.sum, residual_initial_smaller)))
# Equation 8.5 from Horne
var_init_removal_lamda=1/np.abs(np.sum(np.array(Px**2/(var_inputimage_smaller)),axis=1))
std_init_removal_lamda=np.sqrt(var_init_removal_lamda)
#return init_lamda,std_init_lamda,init_removal_lamda,std_init_removal_lamda



NameError: name 'box20' is not defined

In [518]:
plt.figure(figsize=(10,10))
plt.errorbar(np.array(range(len(init_lamda)))-0.15,init_lamda,yerr=std_init_lamda,fmt='o',elinewidth=2,capsize=12,markeredgewidth=2,label='data',color='orange')
plt.errorbar(np.array(range(len(init_removal_lamda)))-0.15,init_removal_lamda,yerr=std_init_removal_lamda,color='red',fmt='o',elinewidth=2,capsize=10,markeredgewidth=2,label='residual')
plt.legend(loc=2, fontsize=22)
plt.plot(np.zeros(20),'--')
plt.ylim(-5000,5000)
plt.ylabel('flux',size=25)
plt.xlabel('pixel',size=25)
plt.xticks(range(20))

for i in range(20):
   plt.text(-0.8+i, -1250, str("{:1.0f}".format(init_lamda[i])), fontsize=20,rotation=60.,color='orange')
    
for i in range(20):
    plt.text(-0.8+i, -2050, str("{:1.1f}".format(init_removal_lamda[i]/std_init_removal_lamda[i])), fontsize=20,rotation=60.,color='red')

sci_image_40000,var_image_40000,res_iapetus_40000=add_artificial_noise(box20,var_20, psf20)
init_lamda,std_init_lamda,init_removal_lamda,std_init_removal_lamda=residual_1D(sci_image_40000,var_image_40000,res_iapetus_40000)

position_of_max_flux=np.where(init_lamda==np.max(init_lamda))[0][0]
difference_from_max=range(40)-position_of_max_flux
pixels_to_test=np.array(range(40))[(np.abs(difference_from_max)>2)&(np.abs(difference_from_max)<=6)]
Q=np.mean(np.abs(init_removal_lamda[pixels_to_test]/std_init_removal_lamda[pixels_to_test]))

plt.text(19.5,2300, 'Q='+str("{:1.2f}".format(Q)), horizontalalignment='right', verticalalignment='top',fontsize=26)

chi2=np.mean((psf20-box20)**2/var_20)

plt.text(19.5,2000, '$\chi^{2}$='+str("{:1.2f}".format(chi2)), horizontalalignment='right', verticalalignment='top',fontsize=26)

chi2_40000=np.mean((res_iapetus_40000-sci_image_40000)**2/var_image_40000)

plt.text(19.5,1700, '$\chi^{2}_{40000}$='+str("{:1.2f}".format(chi2_40000)), horizontalalignment='right', verticalalignment='top',fontsize=26)


plt.axvspan(pixels_to_test[0]-0.5, pixels_to_test[3]+0.5, alpha=0.3, color='grey')
plt.axvspan(pixels_to_test[4]-0.5, pixels_to_test[7]+0.5, alpha=0.3, color='grey')

FigureCanvasNbAgg()

/Users/naka.keigo/lsst_stack/python/miniconda3-4.3.21/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in sqrt


In [389]:
for s in result.sourceCat[::40]:
    im = psf.computeImage(s.getCentroid())
    box=exposure.getImage()[im.getBBox()].convertF()
    psfplt= psf.computeImage(afwGeom.PointD(s.getCentroid())).convertF()
    
    normbox = (box.array)/s.getApFlux()
    normpsf = psfplt.array

    box20=normbox[10:30,10:30]
    psf20=normpsf[10:30,10:30]

    sigma = box20-psf20
    
    plt.figure(figsize=(6,5))
    plt.imshow((box20-psf20)/np.sqrt(var_20),origin='lower',cmap='bwr')
    plt.colorbar()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

...

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [500]:
#plt.xlim(-0.05,0.05)
px = []
py = []
sigma = []

for s in result.sourceCat:
    ix=s.getCentroid()[0]
    iy=s.getCentroid()[1]
    
    px.append(ix)
    py.append(iy)

    im = psf.computeImage(s.getCentroid())
    box=exposure.getImage()[im.getBBox()].convertF()
    psfplt= psf.computeImage(afwGeom.PointD(s.getCentroid())).convertF()
    
    normbox = (box.array[10:30,10:30])/s.getApFlux()
    normpsf = psfplt.array[10:30,10:30]

    sigma.append(((normbox-normpsf)/np.sqrt(var_20)).mean())

plt.figure(figsize=(6,5))
plt.scatter(px, py, c=sigma, cmap='Blues')    
plt.colorbar()

FigureCanvasNbAgg()

In [496]:
#plt.xlim(-0.05,0.05)
px = []
py = []
sigma = []

tilx = []
tily = []

for s in result.sourceCat:
    ix=s.getCentroid()[0]
    iy=s.getCentroid()[1]
    
    px.append(ix)
    py.append(iy)

    im = psf.computeImage(s.getCentroid())
    box=exposure.getImage()[im.getBBox()].convertF()
    psfplt= psf.computeImage(afwGeom.PointD(s.getCentroid())).convertF()
    
    normbox = (box.array[10:30,10:30])/s.getApFlux()
    normpsf = psfplt.array[10:30,10:30]

    sigma.append(((normbox-normpsf)*(normbox-normpsf)).mean())

plt.figure(figsize=(6,5))
plt.scatter(px, py, c=sigma, cmap='Blues')    
plt.colorbar()

FigureCanvasNbAgg()

In [34]:
def residual_1D(sci_image,var_image,res_iapetus):

    """

    @param[in] sci_image        data (20x20 cutout)
    @param[in] var_image        Variance data (20x20 cutout)
    @param[in] res_iapetus      model (20x20 cutout)
    """

    #sci_image =np.load(STAMPS_FOLDER+'sci'+str(obs)+str(single_number)+str(arc)+'_Stacked.npy')
    #var_image =np.load(STAMPS_FOLDER+'var'+str(obs)+str(single_number)+str(arc)+'_Stacked.npy')
    multiplicative_factor_to_renormalize_to_50000=np.max(sci_image)/50000
    sci_image_smaller=sci_image[:,8:14]/multiplicative_factor_to_renormalize_to_50000
    var_image_smaller=var_image[:,8:14]/multiplicative_factor_to_renormalize_to_50000

    residual_initial_smaller=sci_image_smaller-res_iapetus[:,8:14]/multiplicative_factor_to_renormalize_to_50000
    #residual_RF_smaller=chi_RF_corrected_image[:,8:14]*np.sqrt(var_image_smaller)

    #################################
    # step 5 from Horne, very simplified
    inputimage_smaller=sci_image_smaller
    Px=np.sum(inputimage_smaller,axis=0)/np.sum(inputimage_smaller)
    var_inputimage_smaller=var_image_smaller
    #################################
    # Equation 8 from Horne with modification from Robert abut variance for extraction of signal
    # note that this uses profile from full thing, and not "residual profile"

    # nominator
    weighted_inputimage_smaller=inputimage_smaller*Px/(1)
    # denominator
    weights_array=np.ones((inputimage_smaller.shape[0],inputimage_smaller.shape[1]))*Px**2

    init_lamda=np.array(list(map(np.sum, weighted_inputimage_smaller)))/(np.array(list(map(np.sum,weights_array))))
    init_lamda_boxcar=np.array(list(map(np.sum, inputimage_smaller)))
    # Equation 8.5 from Horne
    var_f_std_lamda=1/np.sum(np.array(Px**2/(var_inputimage_smaller)),axis=1)
    std_init_lamda=np.sqrt(var_f_std_lamda)
    std_init_lamda_boxcar=np.sqrt(np.array(list(map(np.sum, var_inputimage_smaller))))


    #################################
    # Equation 8 from Horne with modification from Robert abut variance for initial removal
    # note that this uses profile from full thing, and not "residual profile"

    # nominator
    weighted_inputimage_smaller=residual_initial_smaller*Px/(1)
    # denominator
    weights_array=np.ones((residual_initial_smaller.shape[0],residual_initial_smaller.shape[1]))*Px**2

    init_removal_lamda=np.array(list(map(np.sum, weighted_inputimage_smaller)))/(np.array(list(map(np.sum,weights_array))))
    init_removal_lamda_boxcar=np.array(list(map(np.sum, residual_initial_smaller)))
    # Equation 8.5 from Horne
    var_init_removal_lamda=1/np.sum(np.array(Px**2/(var_inputimage_smaller)),axis=1)
    std_init_removal_lamda=np.sqrt(var_init_removal_lamda)
    return init_lamda,std_init_lamda,init_removal_lamda,std_init_removal_lamda

def chi_50000(sci_image,var_image,res_iapetus):

    #sci_image =np.load(STAMPS_FOLDER+'sci'+str(obs)+str(single_number)+str(arc)+'_Stacked.npy')
    #var_image =np.load(STAMPS_FOLDER+'var'+str(obs)+str(single_number)+str(arc)+'_Stacked.npy')
    multiplicative_factor_to_renormalize_to_50000=np.max(sci_image)/50000
    sci_image_renormalized=sci_image/multiplicative_factor_to_renormalize_to_50000
    var_image_renormalized=var_image/multiplicative_factor_to_renormalize_to_50000
    res_iapetus_renormalized=res_iapetus/multiplicative_factor_to_renormalize_to_50000


    return np.mean((sci_image_renormalized-res_iapetus_renormalized)**2/var_image_renormalized)

In [35]:
def add_artificial_noise(sci_image,var_image,res_iapetus):
    
    """
    add extra noise so that it has comparable noise as if the max flux in the image (in the single pixel) is 40000
    
    
    """

    
    multi_factor=np.max(sci_image)/40000
    Max_SN_now=np.max(sci_image)/np.max(np.sqrt(var_image))
    dif_in_SN=Max_SN_now/200
    artifical_noise=np.zeros_like(res_iapetus)
    artifical_noise=np.array(artifical_noise)
    for i in range(len(artifical_noise)):
        for j in range(len(artifical_noise)):
            artifical_noise[i,j]=np.random.randn()*np.sqrt((dif_in_SN**2-1)*var_image[i,j])   
            
    if dif_in_SN>1:        
        return (sci_image+artifical_noise),((dif_in_SN**2)*var_image),res_iapetus
    else:
        return (sci_image),((dif_in_SN**2)*var_image),res_iapetus 

In [ ]:
[_ for _ in dir(result.sourceCat[100]) if _.endswith("lux")]
s = result.sourceCat[100]
print (s.getPsfFlux())
#dir(result.sourceCat[100])